In [ ]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Define Control File

In [ ]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

## Read From Control File

In [ ]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

## Read in Data

In [ ]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)

## Tier Assignments

#### Thresholds

In [ ]:
tier1_tier2_thresh = 0.1
tier2_tier3_thresh = 0.4

#### Variables

In [ ]:
# USING BOTH S_SLUIS_CVP_ and S_SLUIS_SWP_
# NOTE that S_MELON has only LEVEL4, and that S_MLRTN is a constant
# variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MELON_", "S_MLRTN_"]
# variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MELON_LEVEL5DV"]
variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MLRTN_", "S_MELON_"]
variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL4DV", "S_SLUIS_CVPLEVEL4DV", "S_SLUIS_SWPLEVEL4DV", "S_MLRTNLEVEL4DV", "S_MELONLEVEL4DV"]

##### Helper Function

In [ ]:
def classify_tier(prob, thresh1 = 0.10, thresh2 = 0.40):
    if np.isnan(prob):
        return np.nan
    elif prob < thresh1:
        return "1"
    elif prob < thresh2:
        return "2"
    else:
        return "3"

In [ ]:
# hitting floodpool = variables_storage >= variables_floodpool
#FOLSOM is Nan for scenario 6 - 10 because it doesn't have level5D for sceanrio 6-10 "S_FOLSMLEVEL5DV"
S_MLRTN_level5constant = 524

prob_result_list = []

for scenario in dss_names:
    scenario_short = scenario[:5]
    row_data = {"Scenario": scenario_short}

    for var_s, var_lvl5D in zip(variables_storage, variables_floodpool):
        var1_name = f"{var_s}{scenario_short}"
        var2_name = f"{var_lvl5D}_{scenario_short}"

        p_gte = probability_var1_gte_var2_for_scenario(df, var1_name, var2_name, units="TAF", tolerance=1e-6)
        if(var_s == "S_MLRTN_"):
            p_gte = probability_var1_gte_const_for_scenario(df, var1_name, S_MLRTN_level5constant, units="TAF")
        
        gte_col = f"Prob_{var_s}GTE_{var_lvl5D}"
        tier_col = f"{var_s}FloodTier"

        row_data[gte_col] = p_gte
        row_data[tier_col] = classify_tier(p_gte, tier1_tier2_thresh, tier2_tier3_thresh)

    prob_result_list.append(row_data)


df_flow_probs = pd.DataFrame(prob_result_list)
df_flow_probs.set_index("Scenario", inplace=True)

display(df_flow_probs)

In [ ]:
output_dir = os.path.join(ScenarioDir, "Performance_Metrics", "Tiered_Outcome_Measures", "Reservoir_FloodRisk")
output_path = os.path.join(output_dir, "tier_assignment_output.csv")

# check if output directory exists
if not os.path.exists(output_dir):
    # print warning
    print("Warning: directory " + output_dir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(output_dir)
    
df_flow_probs.to_csv(output_path)

In [ ]:
print("Done!")